In [6]:
#importing libraries
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
import matplotlib.animation as animation
#defining a class for matrix factorisation
class MatFactor():
    #overiding the init function by adding the following parameters as arguments (the function self, Ratings, number of workers, learning rates/weights, #iterations)
    def __init__(self, R, K, alpha, beta, iterations):
        #assigning the values of variables 
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

        # real user as x, item as y and rating as R
        #we need to predict the values in R that aren't assigned by using the non-zero values in R
        xs, ys = self.R.nonzero()
        self.z = [self.R[x, y] for x, y in zip(xs, ys)]
        self.x = [0 for i in range(len(self.z))]
        self.y = [i for i in range(len(self.z))]
        self.realLine = np.array([self.x, self.y, self.z])
        self.log = [self.realLine]
        self.shiftXIndex = 0

    def getShiftX(self):
        self.shiftXIndex += .5
        return [self.shiftXIndex for i in range(len(self.z))]

    def train(self):
        # Initialize user and item latent feature matrix
        self.P = np.random.normal(
            scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(
            scale=1./self.K, size=(self.num_items, self.K))

        # Initialize the biases
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        # general_mean of non zero values
        self.b = np.mean(self.R[np.where(self.R != 0)])

        # Create a list of training samples
        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]

        # Perform stochastic gradient descent for number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            print("Iteration: %d ; error = %.4f" % (i+1, mse))

        return training_process
    #defining function for mean square error
    def mse(self):
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        newPredictLine = np.array(
            [self.getShiftX(), self.y, [predicted[x, y] for x, y in zip(xs, ys)]])

        self.log.append(newPredictLine)
        error = 0
        
        #using for loop for calculating mean square error
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)
    
    #function for performing stochastic gradient descent
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_rating(i, j)
            e = (r - prediction)
            self.b_u[i] += self.alpha * (2*e - self.beta * self.b_u[i]) #PERFORM UPDATION
            self.b_i[j] += self.alpha * (2*e - self.beta * self.b_i[j])

            self.P[i, :] += self.alpha * \
                (e * self.Q[j, :] - self.beta * self.P[i, :])
            self.Q[j, :] += self.alpha * \
                (e * self.P[i, :] - self.beta * self.Q[j, :])

    def get_rating(self, i, j):
        prediction = self.b + self.b_u[i] + \
            self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    def full_matrix(self):
        return self.b + self.b_u[:, np.newaxis] + self.b_i[np.newaxis:, ] + self.P.dot(self.Q.T)


R = np.random.randint(6, size=(5, 5)) #GENERATE Ratings at random
mf = MatFactor(R, K=2, alpha=0.1, beta=0.01, iterations=100) #TRIAL-AND-ERROR FOR THESE HYPERPARAMETERS

mf.train()

Iteration: 1 ; error = 4.7974
Iteration: 2 ; error = 4.3509
Iteration: 3 ; error = 3.4138
Iteration: 4 ; error = 2.7751
Iteration: 5 ; error = 2.3856
Iteration: 6 ; error = 2.0542
Iteration: 7 ; error = 1.8083
Iteration: 8 ; error = 1.5854
Iteration: 9 ; error = 1.3549
Iteration: 10 ; error = 1.1946
Iteration: 11 ; error = 0.9581
Iteration: 12 ; error = 0.7756
Iteration: 13 ; error = 0.6333
Iteration: 14 ; error = 0.4947
Iteration: 15 ; error = 0.4250
Iteration: 16 ; error = 0.3744
Iteration: 17 ; error = 0.3562
Iteration: 18 ; error = 0.3079
Iteration: 19 ; error = 0.2972
Iteration: 20 ; error = 0.2904
Iteration: 21 ; error = 0.2799
Iteration: 22 ; error = 0.2679
Iteration: 23 ; error = 0.2949
Iteration: 24 ; error = 0.2819
Iteration: 25 ; error = 0.2689
Iteration: 26 ; error = 0.2601
Iteration: 27 ; error = 0.2570
Iteration: 28 ; error = 0.2682
Iteration: 29 ; error = 0.2764
Iteration: 30 ; error = 0.2770
Iteration: 31 ; error = 0.2476
Iteration: 32 ; error = 0.2785
Iteration: 33 ; e

[(0, 4.797378500696788),
 (1, 4.350850007942864),
 (2, 3.413835364467555),
 (3, 2.7751171382961335),
 (4, 2.3855512095787725),
 (5, 2.0541702493122957),
 (6, 1.8082520878869468),
 (7, 1.5854033433628856),
 (8, 1.35489638874836),
 (9, 1.1945649105850746),
 (10, 0.9581458741763222),
 (11, 0.7755559191717589),
 (12, 0.6332562253697951),
 (13, 0.494677465171062),
 (14, 0.425007351759414),
 (15, 0.3744466219568634),
 (16, 0.35619609606615177),
 (17, 0.3078563541643459),
 (18, 0.2972318344094391),
 (19, 0.29043613009673497),
 (20, 0.2799032149947495),
 (21, 0.26789950359321096),
 (22, 0.2948640647035681),
 (23, 0.28186945535927654),
 (24, 0.26889566145580374),
 (25, 0.26014449404421297),
 (26, 0.2570112268195403),
 (27, 0.2681514606590825),
 (28, 0.2764433863136213),
 (29, 0.27703424864496695),
 (30, 0.24763565029469709),
 (31, 0.27853450122827234),
 (32, 0.24590810414444325),
 (33, 0.24098000240979414),
 (34, 0.2535597621901794),
 (35, 0.2606798249031276),
 (36, 0.2353039120644176),
 (37, 0